In [ ]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import string
import unicodedata
# Приєднуєиося до сторінки flatfy
HEADERS ={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}
olx_link = requests.get('https://flatfy.ua/search?geo_id=1&page=1&section_id=2')
olx_link.encoding = "utf-8"
soup = BeautifulSoup(olx_link.text, 'html.parser')
prod_div = soup.find('div', "table-view-wrap").find_all('article', 'realty-preview')
data_list = []

# Проходимо по сторінці

for i in prod_div:
    listinlist = []
    listinlist.append(int(((i.find('div', 'realty-preview-price realty-preview-price--main').text).split(" ")[0]).replace(u'\xa0',''))) #Ціна
    listinlist.append(float(i.find('div', 'realty-preview-properties-item').text.split(' ')[0])) #К-кість кімнат
    listinlist.append(float(i.find('div', 'realty-preview-properties-item').findNext('div').text.split(' ')[0])) #Площа
    listinlist.append(i.find('a', 'realty-preview-sub-title').findNext('a').text.split(' ')[0]) #Район
    listinlist.append(i.find('h3', 'realty-preview-title').text) #Адреса
    build_type = i.find('a', 'realty-preview-sub-title').text.split(' ')[0]
    if build_type == "ЖК" or build_type == "ЖМ":
        listinlist.append(1)
    else:
        listinlist.append(0) #ЖК чи ні
    card_link = requests.get("https://flatfy.ua/uk/" + str(i.find('a', target='_blank').get('href'))[4:], allow_redirects=True, headers=HEADERS) # переходимо по посиланню з карточки
    card_link.encoding = "utf-8"
    soup2 = BeautifulSoup(card_link.text, 'html.parser')
    card_link = requests.get(soup2.find('a', 'redirect-link').get('href')) # отримуємо лінк карточки з редиректу
    print(card_link.url) # первірка посилання
    soup2 = BeautifulSoup(card_link.text, 'html.parser')
    #listinlist.append(soup2.find('div','offer-view-details-column').findNext('div','offer-view-details-column').findNext('span').text) !!! тут тупняк почався
    data_list.append(listinlist)

print(data_list)